In [40]:
### importing libraries
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import bs4
from langchain_huggingface import HuggingFaceEmbeddings

In [41]:
### loading environment variables
load_dotenv()
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [8]:
### initializing llm model
llm=Ollama(model="gemma3:1b")

C:\Users\vamsi\AppData\Local\Temp\ipykernel_13452\1263823765.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="gemma3:1b")


In [43]:
### initializing Embedding model
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\main_project_space\generative_ai_projects\Q&A_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
### load, chunk and create indexes of this blog to create a retriever
loader=WebBaseLoader(
    web_paths=("https://www.cricbuzz.com/cricket-news/135464/bangladesh-cricket-board-news-bcb-elections-set-for-early-october-amid-conflict-of-interest-concerns",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("cb-nws-para")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://www.cricbuzz.com/cricket-news/135464/bangladesh-cricket-board-news-bcb-elections-set-for-early-october-amid-conflict-of-interest-concerns'}, page_content=' Bangladesh Cricket Board announced on Monday (September 1) that the upcoming elections of the board will be held in the first week of October. Cricbuzz had earlier reported that a decision regarding BCB election will be taken following the board meeting that took place at Sylhet earlier today.  "The BCB election will be held in the first week of October. The election commission will confirm the exact date," BCB vice-president Nazmul Abedin told reporters following the meeting. "The Election Commission has not yet been formed by name. The responsibility of recommending names has been left to the President. A three-member Election Commission will be formed, and I believe we will know about it within a couple of days. We will follow the directions from the constitution," he said.  Though the date 

In [33]:
### chunking documents
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [34]:
splits

[Document(metadata={'source': 'https://www.cricbuzz.com/cricket-news/135464/bangladesh-cricket-board-news-bcb-elections-set-for-early-october-amid-conflict-of-interest-concerns'}, page_content='Bangladesh Cricket Board announced on Monday (September 1) that the upcoming elections of the board will be held in the first week of October. Cricbuzz had earlier reported that a decision regarding BCB election will be taken following the board meeting that took place at Sylhet earlier today.  "The BCB election will be held in the first week of October. The election commission will confirm the exact date," BCB vice-president Nazmul Abedin told reporters following the meeting. "The Election Commission has not yet been formed by name. The responsibility of recommending names has been left to the President. A three-member Election Commission will be formed, and I believe we will know about it within a couple of days. We will follow the directions from the constitution," he said.  Though the date i

In [45]:
### initializing vectordatabase
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000026C9B44C180>, search_kwargs={})

In [46]:
### prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
### creating chain
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [49]:
response=rag_chain.invoke({"input":"what this Blog telling about?"})
response

{'input': 'what this Blog telling about?',
 'context': [Document(id='aeec4dd2-c72e-44ee-83e0-80eb3d41863f', metadata={'source': 'https://www.cricbuzz.com/cricket-news/135464/bangladesh-cricket-board-news-bcb-elections-set-for-early-october-amid-conflict-of-interest-concerns'}, page_content='be present, to do a skill programme for our local coaches. We are very excited about it, as this is the first time we will see such an event in Bangladesh," he said.  "We will also organize a level-3 coaching course in Bangladesh. Australian educators, like Ashley Ross, will conduct the course. It will be a high-profile programme. Those who haven\'t completed a level-3 coaching course, will participate in the programme. It could be around 15-20 coaches," he added.  Meanwhile, Julian Wood\'s contract will not be extended beyond September 3. If needed later, he will be brought back again.  The meeting was chaired by the BCB boss Aminul while some board directors joined the meeting through Zoom. Howeve

In [50]:
response['answer']

'The text is telling about an upcoming event in Bangladesh where a series of skill programmes for local coaches will be held, including a level-3 coaching course and an international batting programme. There’s also a potential election for the BCB president, and a discussion about potential conflicts of interest.'